In [27]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import os
import numpy as np
import random
import string
import tensorflow as tf
import zipfile
from six.moves import range
from six.moves.urllib.request import urlretrieve

In [28]:
url = 'http://mattmahoney.net/dc/'

def maybe_download(filename, expected_bytes):
  """Download a file if not present, and make sure it's the right size."""
  if not os.path.exists(filename):
    filename, _ = urlretrieve(url + filename, filename)
  statinfo = os.stat(filename)
  if statinfo.st_size == expected_bytes:
    print('Found and verified %s' % filename)
  else:
    print(statinfo.st_size)
    raise Exception(
      'Failed to verify ' + filename + '. Can you get to it with a browser?')
  return filename

filename = maybe_download('text8.zip', 31344016)

Found and verified text8.zip


In [29]:
def read_data(filename):
  with zipfile.ZipFile(filename) as f:
    name = f.namelist()[0]
    data = tf.compat.as_str(f.read(name))
  return data
  
text = read_data(filename)
print('Data size %d' % len(text))

Data size 100000000


In [30]:
valid_size = 1000
valid_text = text[:valid_size]
train_text = text[valid_size:]
train_size = len(train_text)
print(train_size, train_text[:64])
print(valid_size, valid_text[:64])

99999000 ons anarchists advocate social relations based upon voluntary as
1000  anarchism originated as a term of abuse first used against earl


In [31]:
import collections
def read_data_as_bigrams(filename):
  """Extract the first file enclosed in a zip file as a list of bigrams"""
  with zipfile.ZipFile(filename) as f:
    text = tf.compat.as_str(f.read(f.namelist()[0]))
    data = []
    lenght = len(text)
    #create bigrams
    for i in range(lenght - 1):
        data.append(text[i] + text[i+1])
  return data

def build_bigram_dataset(words):
  count = [['UNK', -1]]
  count.extend(collections.Counter(words).most_common(50000 - 1))
  dictionary = dict()
  for word, _ in count:
    dictionary[word] = len(dictionary)
  data = list()
  unk_count = 0
  for word in words:
    if word in dictionary:
      index = dictionary[word]
    else:
      index = 0  # dictionary['UNK']
      unk_count = unk_count + 1
    data.append(index)
  count[0][1] = unk_count
  reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys())) 
  return data, count, dictionary, reverse_dictionary

words = read_data_as_bigrams('text8.zip')
data, count, bigram_dictionary, bigram_reverse_dictionary = build_bigram_dataset(words)
bigram_vocabulary_size = len(count)
print("Number of unique bigrams: {0}".format(len(count)))
print('Most common bigrams (+UNK)', count[:20])
print('Sample data', data[:10])
del words  # Hint to reduce memory.

Number of unique bigrams: 729
Most common bigrams (+UNK) [['UNK', 0], ('e ', 3686256), (' t', 2449254), ('s ', 2222333), ('th', 1980538), (' a', 1846251), ('in', 1717706), ('n ', 1654644), ('he', 1597274), ('er', 1549252), (' o', 1468881), ('d ', 1442521), ('on', 1430301), (' s', 1296911), ('an', 1293588), ('t ', 1254402), ('re', 1121783), (' i', 1066337), ('ne', 1032713), ('r ', 1006225)]
Sample data [5, 14, 97, 31, 218, 79, 75, 33, 267, 92]


In [32]:
batch_size=64
num_unrollings=20

class BigramBatchGenerator(object):
  def __init__(self, text, batch_size, num_unrollings):
    self._text = text
    self._text_size = len(text)
    self._batch_size = batch_size
    self._num_unrollings = num_unrollings
    segment = self._text_size // (2*batch_size)
    self._cursor = [ offset * segment for offset in range(batch_size)]
    self._last_batch = self._next_batch()
  
  def _next_batch(self):
    """Generate a single batch from the current cursor position in the data."""
    
    # Generate a batch taking only two chars (bigram) from each segment. Next batch will take the
    # following chars in each segment and so on.
    batch = np.zeros(shape=(self._batch_size), dtype=np.int32)
    for b in range(self._batch_size):
      # We lookup the corresponding bigram id and add it to the batch
      batch[b] = bigram_dictionary[self._text[self._cursor[b]]+self._text[self._cursor[b] + 1]] 
      # Increment segment cursor +2 so the next batch picks the next bigram of each segment
      self._cursor[b] = (self._cursor[b] + 2) % self._text_size
    return batch
  
  def next(self):
    """Generate the next array of batches from the data. The array consists of
    the last batch of the previous array, followed by num_unrollings new ones.
    """
    batches = [self._last_batch]
    for step in range(self._num_unrollings):
      batches.append(self._next_batch())
    self._last_batch = batches[-1]
    return batches

def bigrams(ids):
  """Turn a 1-hot encoding or a probability distribution over the possible
  bigrams back into its (most likely) character representation."""
  return [bigram_reverse_dictionary[c] for c in ids]

def bigram_batches2string(batches):
  """Convert a sequence of batches back into their (most likely) string
  representation."""
  
  # To reconstruct a word, we need to pick the bigrams in the same position of each batch
  # and concatenate them.
  # On the first iteration, the comprehension simply transforms ids into bigrams and 
  # copies them into s. On every subsequent iteration the comprehension will concatenate
  # what's already in s with what's in the next batch in the same position.
  # In this way it reconstruct the words by joining together the bigrams that were in 
  # different batches
  # We have n batches with m bigrams on each. This will generate m strings with n bigrams on each
  s = [''] * batches[0].shape[0]
  for b in batches:
    
    s = [''.join(x) for x in zip(s, bigrams(b))]
  return s

train_batches = BigramBatchGenerator(train_text, batch_size, num_unrollings)
valid_batches = BigramBatchGenerator(valid_text, 1, 1)
print(bigram_batches2string(train_batches.next()))
print(bigram_batches2string(train_batches.next()))
print(bigram_batches2string(valid_batches.next()))
print(bigram_batches2string(valid_batches.next()))

['ons anarchists advocate social relations b', 'nomination gore s endorsement of dean was ', 'when military governments failed to revive', ' three nine one six zero two zero zero one', 'lleria arches national park photographic v', 'reviated as dr mr and mrs respectively the', ' abbeys and monasteries index sacred desti', 'shing the right of appeal to the judicial ', 'married urraca princess of castile daughte', 'sity upset the devils which cost the schoo', 'hel and richard baer h provided a detailed', 'ased in the st family here they are in rou', 'y and liturgical language among jews manda', ' disgust because of the relationship betwe', 'ay opened for passengers in december one n', 'society and that this neglect is the true ', 'tion from the national media and from pres', 'ago based chess records label the influenc', 'migration took place during the one nine e', ' zero zero five yaniv shaked and avishai w', 'new york other well known manufacturers of', 'short subject college humor one 

In [33]:
def logprob(predictions, labels):
  """Log-probability of the true labels in a predicted batch."""
  predictions[predictions < 1e-10] = 1e-10
  return np.sum(np.multiply(labels, -np.log(predictions))) / labels.shape[0]

def sample_distribution(distribution):
  """Sample one element from a distribution assumed to be an array of normalized
  probabilities.
  """
  r = random.uniform(0, 1)
  s = 0
  for i in range(len(distribution)):
    s += distribution[i]
    if s >= r:
      return i
  return len(distribution) - 1

def sample_bigram(prediction):
  """Turn a (column) prediction into 1-hot encoded samples."""
  p = np.zeros(shape=[1, bigram_vocabulary_size], dtype=np.float)
  p[0, sample_distribution(prediction[0])] = 1.0
  return p

def bigram_random_distribution():
  """Generate a random column of probabilities."""
  b = np.random.uniform(0.0, 1.0, size=[1, bigram_vocabulary_size])
  return b/np.sum(b, 1)[:,None]

In [34]:
num_nodes = 128
graph = tf.Graph()
with graph.as_default():
    
  
  # Parameters:
  x = dict()
  m = dict()
  bl = dict()
  # Layer 1
  x["layer 1"] = tf.Variable(tf.truncated_normal([bigram_vocabulary_size, 4*num_nodes], -0.1, 0.1))
  m["layer 1"] = tf.Variable(tf.truncated_normal([num_nodes, 4*num_nodes], -0.1, 0.1))
  bl["layer 1"] = tf.Variable(tf.zeros([1, 4*num_nodes]))
  # Layer 2
  x["layer 2"] = tf.Variable(tf.truncated_normal([num_nodes, 4*num_nodes], -0.1, 0.1))
  m["layer 2"] = tf.Variable(tf.truncated_normal([num_nodes, 4*num_nodes], -0.1, 0.1))
  bl["layer 2"] = tf.Variable(tf.zeros([1, 4*num_nodes]))
  
  # Variables saving state across unrollings for each layer.
  saved_output = dict()
  saved_state = dict()
  # Layer 1
  saved_output["layer 1"] = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  saved_state["layer 1"] = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  # Layer 2
  saved_output["layer 2"] = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  saved_state["layer 2"] = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  
  
  # Classifier weights and biases.
  w = tf.Variable(tf.truncated_normal([num_nodes, bigram_vocabulary_size], -0.1, 0.1))
  b = tf.Variable(tf.zeros([bigram_vocabulary_size]))
  
  # Definition of the cell computation.
  def lstm_cell(i, o, state, layer):
    """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
    Note that in this formulation, we omit the various connections between the
    previous state and the gates."""
    
    # the lookup is equivalent to matrix multipication between x and a one-hot encoded input vector
    if layer == "layer 1":
      lstm_tensor = tf.nn.embedding_lookup(x[layer], i) + tf.matmul(o, m[layer]) + bl[layer]
    else:
      lstm_tensor = tf.matmul(i, x[layer]) + tf.matmul(o, m[layer]) + bl[layer]
    
    input_gate = tf.sigmoid(lstm_tensor[:, :num_nodes])
    forget_gate = tf.sigmoid(lstm_tensor[:, num_nodes:2*num_nodes])
    update = lstm_tensor[:, 2*num_nodes:3*num_nodes]
    output_gate = tf.sigmoid(lstm_tensor[:, 3*num_nodes:])
    
    state = forget_gate * state + input_gate * tf.tanh(update)
    return output_gate * tf.tanh(state), state

  # Input data. Now instead of one-hot encoding vectors, we only take the bigram id as input
  train_data = list()
  #train_labels = list()
  for _ in range(num_unrollings + 1):
    train_data.append(
      tf.placeholder(tf.int32, shape=[batch_size]))
    
  train_inputs = train_data[:num_unrollings]
    
  identity = tf.constant(np.identity(bigram_vocabulary_size, dtype = np.int32))
  train_labels = [tf.nn.embedding_lookup(identity, td) for td in train_data[1:]]
    
  # Unrolled LSTM loop.
  outputs = list()
  output = saved_output["layer 1"]
  state = saved_state["layer 1"]
  output_layer_2 = saved_output["layer 2"]
  state_layer_2 = saved_state["layer 2"]
    
  for i in train_inputs:
    output, state = lstm_cell(i, output, state, "layer 1")
    #outputs.append(tf.nn.dropout(output, keep_prob=0.8))
    output_layer_2, state_layer_2 = lstm_cell(output, output_layer_2, state_layer_2, "layer 2")
    outputs.append(output_layer_2)

  # State saving across unrollings.
  with tf.control_dependencies([saved_output["layer 1"].assign(output),
                                saved_state["layer 1"].assign(state),
                                saved_output["layer 2"].assign(output_layer_2),
                                saved_state["layer 2"].assign(state_layer_2)]):
    
    
    # Classifier.
    logits = tf.nn.xw_plus_b(tf.concat(outputs, 0), w, b)
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(
        labels=tf.concat(train_labels, 0), logits=logits))

  # Optimizer.
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(
    10.0, global_step, 5000, 0.1, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  gradients, v = zip(*optimizer.compute_gradients(loss))
  gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
  optimizer = optimizer.apply_gradients(
    zip(gradients, v), global_step=global_step)

  # Predictions.
  train_prediction = tf.nn.softmax(logits)
  
  # Sampling and validation eval: batch 1, no unrolling.
  sample_input = tf.placeholder(tf.int32, shape=[1])
  saved_sample_output = dict()  
  saved_sample_state = dict()
  saved_sample_output["layer 1"] = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_state["layer 1"] = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_output["layer 2"] = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_state["layer 2"] = tf.Variable(tf.zeros([1, num_nodes]))
  reset_sample_state = tf.group(
    saved_sample_output["layer 1"].assign(tf.zeros([1, num_nodes])),
    saved_sample_state["layer 1"].assign(tf.zeros([1, num_nodes])),
    saved_sample_output["layer 2"].assign(tf.zeros([1, num_nodes])),
    saved_sample_state["layer 2"].assign(tf.zeros([1, num_nodes])))

  sample_output = dict()
  sample_state = dict()
  sample_output["layer 1"], sample_state["layer 1"] = lstm_cell(
    sample_input, saved_sample_output["layer 1"], saved_sample_state["layer 1"], "layer 1")
    
  sample_output["layer 2"], sample_state["layer 2"] = lstm_cell(
    sample_output["layer 1"], saved_sample_output["layer 2"], saved_sample_state["layer 2"], "layer 2")

  with tf.control_dependencies([saved_sample_output["layer 1"].assign(sample_output["layer 1"]),
                                saved_sample_state["layer 1"].assign(sample_state["layer 1"]),
                                saved_sample_output["layer 2"].assign(sample_output["layer 2"]),
                                saved_sample_state["layer 2"].assign(sample_state["layer 2"])]):
        
    sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output["layer 2"], w, b))



In [35]:
num_steps = 7001
summary_frequency = 100

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  mean_loss = 0
  for step in range(num_steps):
    batches = train_batches.next()
    feed_dict = dict()
    for i in range(num_unrollings + 1):
      feed_dict[train_data[i]] = batches[i]
    
    _, l, predictions, lr, batch_labels = session.run(
      [optimizer, loss, train_prediction, learning_rate, train_labels], feed_dict=feed_dict)
    mean_loss += l
    if step % summary_frequency == 0:
      if step > 0:
        mean_loss = mean_loss / summary_frequency
      # The mean loss is an estimate of the loss over the last few batches.
      print(
        'Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))
      mean_loss = 0
      print('Minibatch perplexity: %.2f' % float(
        np.exp(logprob(predictions, np.reshape(batch_labels,[-1, bigram_vocabulary_size])))))
      if step % (summary_frequency * 10) == 0:
        # Generate some samples.
        print('=' * 40)
        for _ in range(5):
          feed = sample_bigram(bigram_random_distribution())
          sentence = bigrams([np.argmax(feed)])[0]
          reset_sample_state.run()
          for _ in range(39):
            prediction = sample_prediction.eval({sample_input: [np.argmax(feed)]})
            feed = sample_bigram(prediction)
            sentence += bigrams([np.argmax(feed)])[0]
          print(sentence)
        print('=' * 40)
      # Measure validation set perplexity.
      reset_sample_state.run()
      valid_logprob = 0
      for _ in range(valid_size):
        b = valid_batches.next()
        predictions = sample_prediction.eval({sample_input: b[0]})
        valid_label = np.zeros([1, bigram_vocabulary_size])
        valid_label[0,int(b[1])] = 1
        valid_logprob = valid_logprob + logprob(predictions, valid_label)
      print('Validation set perplexity: %.2f' % float(np.exp(
        valid_logprob / valid_size)))

Initialized
Average loss at step 0: 6.619672 learning rate: 10.000000
Minibatch perplexity: 749.70
dzye llemr cnreb hdlvpjrrlxsqxhccbzguauwisxartequjsobozzfhcrxrtwhntibjzsavpqzdlm
t unfhjclzgmzqnwgtjvslq yvavgbtswdwqualefikqmmfrtuszzuvtykpbygkorgezvlaqhucb bcu
kfp hesssc kpgkiqopdbrumurmhzwliarssy sfkkfrqd ysvlzaptyvfmoabkcuipjlmede jarneb
dsfv kmuaxmxfehypmrqyvgpovrmgdazlocvjz miikumpqnbkonhsmtdtmjqghheewrxyasnudocqef
ijvmspdatlvppec  ehygqowofxtzafzvruwvwndoxasmcikgazyexyiehhanmfgdubun eiyufjoezo
Validation set perplexity: 647.07
Average loss at step 100: 5.544343 learning rate: 10.000000
Minibatch perplexity: 203.47
Validation set perplexity: 206.12
Average loss at step 200: 5.316868 learning rate: 10.000000
Minibatch perplexity: 215.71
Validation set perplexity: 193.36
Average loss at step 300: 5.286790 learning rate: 10.000000
Minibatch perplexity: 193.83
Validation set perplexity: 189.73
Average loss at step 400: 5.266559 learning rate: 10.000000
Minibatch perplexity: 179.09
Vali